In [1]:
import os
from Bio.Align.Applications import ClustalwCommandline
from Bio import Entrez 
from Bio import SeqIO 
from Bio.Align.Applications import MuscleCommandline
from Bio import Phylo

import pandas as pd 

def fetch_record(genbank_id): 
    Entrez.email = "dnjs0148@naver.com" 
    handle = Entrez.efetch(db="nucleotide", id=genbank_id, rettype="gb", retmode="xml") 
    records = Entrez.read(handle) 
    return records 

def get_sequence(records): 
    return records[0]["GBSeq_sequence"] 

#=========================== covid.fasta 파일에 바이러스 염기서열을 모두 써준다. =============================
def write_sequence(genbank_id, records): 
    print("## processing: "+genbank_id) 
    record_id = records[0]["GBSeq_accession-version"] 
    record_desc = records[0]["GBSeq_definition"] 
    record_seq = records[0]["GBSeq_sequence"] 
    
    with open("covid.fasta",'a') as fw: 
        fw.write(">"+record_id+" "+record_desc+"\n") 
        fw.write(record_seq+"\n") 
        
def read_table(file): 
    df = pd.read_csv(file, sep="\t") 
    return df 

#============================ number만큼 genbank_id를 입력하고 저장한다. ========================
f = open("covidlist.txt", 'a')
f.write("GenBank"+"\t"+"RefSeq"+"\t"+"Gene Region"+"\n")

a = int(input('number : '))
genbank_id = []
for b in range(0, a):
    genbank_id.append(input('name : '))
    
for b in range(0, a):
    f.write(genbank_id[b]+"\t"+"\t"+"complete"+"\n")

f.close()

#========== txt파일내에 있는 genbank_id에 해당하는 유전자의 염기서열을 covid.fasta 파일에  작성한다. ===========
file = "covidlist.txt" 
df = read_table(file) 
complete_df = df[df["Gene Region"]=="complete"] 
genbank_id_list = complete_df["GenBank"] 
#genbank_id_list = complete_df["GenBank"] 
for genbank_id in genbank_id_list: 
    records = fetch_record(genbank_id) 
    write_sequence(genbank_id, records)

cline = ClustalwCommandline("clustalw2", infile="covid.fasta")
print(cline)

# clustalw_exe = r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\ClustalW2"                                          
clustalw_exe = r"C:\Program Files (x86)\ClustalW2\clustalw2.exe"
clustalw_cline = ClustalwCommandline(clustalw_exe, infile="covid.fasta")
assert os.path.isfile(clustalw_exe), "Clustal W executable missing"

stdout, stderr = clustalw_cline()

f = open('covid.aln', 'r')
while True:
    line = f.readline()
    if not line: break
    print(line)

tree = Phylo.read("covid.dnd", "newick")
Phylo.draw_ascii(tree)

number :  2
name :  MH940245
name :  MH940245


## processing: MH940245
## processing: MH940245
clustalw2 -infile=covid.fasta


ApplicationError: Non-zero return code 4294967295 from '"C:\\Program Files (x86)\\ClustalW2\\clustalw2.exe" -infile=covid.fasta', message 'ERROR: Multiple sequences found with same name (found MH940245.1 at least twice)!'